In [1]:
# EJG Animal Shelter Dashboard Application - Enhanced Version
# Author: Edward Garcia

# Overview:
# This is an enhanced version of my Animal Shelter Dashboard, originally developed for my CS340 course.
# My enhancements aim to improve user experience, functionality, and maintainability of the application through enhanced UX/UI design with logging and 
# testing, optimized data structures and algorithms for searching, caching, and data access. I will also enhance the database aspect of the application 
# by optimizing user data, passwords, roles, security, and authentication.

## Enhancement 1 - Software Design and Engineering
# 1. User Interface and User Experience:
#    - Integrated Dash Bootstrap components to provide better styling and responsiveness.
#    - Applied the VAPOR Bootstrap theme to create a visually appealing and consistent interface.
#    - Improved data presentation by utilizing cards, colors, custom fonts, and shadow effects to headers for a more engaging user interface.

# 2. Navigation and Search Functionality:
#    - Added a responsive navigation bar with links to the About Us and Contact Us sections, enabling easy navigation that scrolls to the page section.
#    - Introduced a search bar in the navigation to filter animal data by breed or name, with modifications to ensure search queries are 
#      case-insensitive for a more user-friendly experience.

# 3. Data Management Enhancements:
#    - Added a "Refresh Data" button, allowing users to easily update and view the most current dataset.
#    - Added a "Download Data" button to provide users with the ability to download the animal dataset as a CSV file directly from the dashboard.

# 4. User Engagement Features:
#    - Included dedicated sections for "About Us" and "Contact Us" to improve user engagement and provide relevant organizational information.

# 5. Logging and Maintainability:
#    - Implemented logging to capture and track dashboard activities, which aids in debugging, application monitoring, and maintaining behavior.

## Enhancement 2 - Data Structures and Algorithms
# 1. Enhanced flexibility with caching bypass:
#    - Introduced a `bypass_cache` parameter in the `read` method to allow direct database queries when live data is required.
#    - Incorporated `bypass_cache` into the dashboard callbacks, ensuring accurate results for user interactions like filtering, map updates, and 
#      downloads.

# 2. Optimized data access:
#    - Implemented a hashable query transformation to enable caching for MongoDB queries, so that it is in sync with `lru_cache`.
#    - Improved search efficiency in callbacks by applying case-insensitive filters for user search input.

# 3. Enhanced search algorithms with testing:
#    - Optimized search and filter functions in the dashboard to improve responsiveness, especially for large datasets.
#    - Used Python’s efficient data handling techniques like `apply` and `lambda` for real-time filtering.
#    - To validate and demonstrate the effectiveness of my enhancements (hash map, LRU caching, binary search),
#      I also implemented several tests within the notebook.

## Enhancement 3 - Databases 
# 1. User Authentication:
#    - Added a new login page with fields for username, password, and MFA OTP to authenticate users which futher enhances the UI/UX.
#    - I Created 3 new users for the AAC database; user1, user2, user3. user1 has MFA enabled and the other 2 users do not.
#      Each user has their own unique password that has been hashed. 
#    - Integrated a logout button in the layout page.
#    - Utilized a structured authentication workflow, including OTP verification and password validation.

# 2. Secure Password Storage:
#    - Used bcrypt for hashing user passwords with salt to guarantee secure storage and mitigate brute force attacks.

# 3. Multi-Factor Authentication (MFA):
#    - Enabled MFA using pyotp to generate time-based one-time passwords (TOTP).
#    - Provided QR codes for users to configure their MFA in authentication apps. I used Microsoft Authenticator. 

# 4. Role-Based Access Control (RBAC):
#    - Implemented roles like guest, regular user, and admin so that user permissions align with their roles.
#    - Verified roles with a `check_permissions` method to restrict access to admin-only features.

# 5. Logging and Monitoring:
#    - Structured logging captures login attempts, role validations, and MFA verification events.

# 6. Testing:
#    - Developed unit tests for authentication, MFA, and RBAC.

# Setup the Jupyter version of Dash
from dash import Dash
import dash
import dash_bootstrap_components as dbc  # Added Dash Bootstrap components for enhanced UI/UX.
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import dash_leaflet as dl
import plotly.express as px
import base64
import pandas as pd
import io
import logging # Here I imported the logging componenent for strucutured loggin of the application. 
from user_management import UserManagement
from user_management import UserManagement, user_management_logger
from dash.exceptions import PreventUpdate  # Add this import at the top of your file

# User Authentication Class Instance
user_mgmt = UserManagement()  


# Global variables for user authentication status
user_authenticated = False  # This tracks if a user is currently authenticated.
current_user = None  # This stores the username of the currently authenticated user.


# Here I added logging to track events, debugging, and monitor application behavior.
logging.basicConfig(
    filename='animal_shelter_dashboard.log',  # File to save logs related to the dashboard
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# Import the AnimalShelter class for CRUD operations
from animal_shelter_CRUD_revised import AnimalShelter

###############################
# Data Manipulation / Model
###############################

# This is the MongoDB username and password I setup for the docker container running MongoDB
username = "edwardgarcia5_snhu"
password = "password"

# Establish a connection to the database using the AnimalShelter class
shelter = AnimalShelter(username, password, 'host.docker.internal', 27017, 'AAC', 'animals')

# Fetch data
data = shelter.read({}, bypass_cache=True) # added in bypass_cache for direct database queries. 
df = pd.DataFrame.from_records(data)
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

###############################
# Dashboard Layout / View
###############################

# This is where I added the Dash Bootstrap theme called VAPOR for a more modern and engaging look.
app = Dash(__name__, external_stylesheets=[dbc.themes.VAPOR], suppress_callback_exceptions=True)

# Here I loaded and encoded the Grazioso Salvare logo to display it in the navigation bar.
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# User authentication flag
user_authenticated = False

# Here I defined the dashboard layout with new navigation and filter options for enhanced interactivity.
app.layout = dbc.Container([

    # Here I added the login section to authenticate users securely with username, password, and MFA OTP.
    html.Div(
        id="login-section",  # This section is dynamically displayed based on user authentication status.
        children=[
            # Centered Logo and Welcome Message
            dbc.Row(
                [
                    # Here I added the Grazioso Salvare logo, centered for a welcoming visual element.
                    dbc.Col(
                        html.Img(
                            src='data:image/png;base64,{}'.format(encoded_image.decode()),
                            style={
                                'width': '150px',  # Set a fixed width for the logo.
                                'marginBottom': '20px',
                                'objectFit': 'contain'  # Maintains the aspect ratio of the logo.
                            }
                        ),
                        width=12,
                        style={'textAlign': 'center'}  # Ensures the logo is centered.
                    ),
                    # Here I added a welcome message for a friendly user interface.
                    dbc.Col(
                        html.H3(
                            "Welcome to EJG's Animal Shelter",
                            style={
                                'color': '#f0e31a',  # Custom color for branding.
                                'fontWeight': 'bold',
                                'marginBottom': '30px',
                                'textAlign': 'center'  # Centers the welcome text.
                            }
                        ),
                        width=12
                    )
                ],
                justify="center",
                style={'marginTop': '20px'}  # Adds spacing above the section for better visual balance.
            ),

            # Login Card
            # Here I added a login card with input fields for username, password, and OTP to authenticate users.
            dbc.Card(
                dbc.CardBody([
                    dbc.Input(id="username", placeholder="Enter username", type="text", className="mb-3"),  # Input field for username.
                    dbc.Input(id="password", placeholder="Enter password", type="password", className="mb-3"),  # Input field for password.
                    dbc.Input(id="otp", placeholder="Enter OTP (if enabled)", type="text", className="mb-3"),  # Added OTP field for MFA.
                    # Login Button
                    # Here I added a login button to trigger the authentication process.
                    dbc.Row(
                        dbc.Col(
                            dbc.Button("Login", id="login-button", color="primary", className="mt-3"),
                            width=12,
                            style={'textAlign': 'center'}  # Center the button within the card.
                        )
                    ),
                    # Error Message Placeholder
                    # Here I added a placeholder to display error messages during login attempts.
                    html.Div(id="login-error", className="text-danger mt-3")  # Displays error messages in red text.
                ]),
                style={'width': '30%', 'margin': 'auto', 'marginTop': '20px'}  # Centered login card with defined width.
            )
        ],
        style={'display': 'block'} if not user_authenticated else {'display': 'none'}  # Show login section only if the user is not authenticated.
    ),

    # Here I added the dashboard section, which is displayed once the user is authenticated.
    html.Div(
        id="dashboard-section",
        children=[
            # Navigation Bar
            # Here I added a responsive navigation bar with branding and interactive features.
            dbc.Navbar(
                dbc.Container([
                    # Here I added the brand name with custom styling for visual appeal.
                    dbc.NavbarBrand(
                        "EJG's Animal Shelter",
                        className="me-2",
                        style={
                            'color': '#f0e31a',  # Custom brand color.
                            'fontWeight': 'bold',
                            'fontSize': '2rem',
                            'textShadow': '2px 2px 4px rgba(0, 0, 0, 0.5)',  # Added shadow for a modern look.
                            'letterSpacing': '1.3px'
                        }
                    ),
                    # Here I added links to the "About Us" and "Contact Us" sections for easy navigation.
                    dbc.Nav(
                        [
                            dbc.NavItem(dbc.NavLink("About Us", href="#about-us", external_link=True)),
                            dbc.NavItem(dbc.NavLink("Contact Us", href="#contact", external_link=True))
                        ],
                        className="ms-auto"
                    ),
                    # Here I added a search input field to filter animal data by breed or name.
                    dcc.Input(id='search-input', type='text', placeholder='Search by breed or name...', debounce=True, style={'width': '300px'}),
                    # Here I added a logout button to allow users to securely log out and reset the session.
                    dbc.Button("Logout", id="logout-button", color="danger", className="ms-3")
                ]),
                color="primary", dark=True, className="mb-4"  # Set the navigation bar to a dark theme for contrast.
            ),

            # Logo Section with Central Placement
            dbc.Row([
                dbc.Col(
                    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height': '10%', 'width': '10%'})),
                    width=12
                )
            ], className="mb-4"),

            # Filter and Data Table Section
            dbc.Row([
                # Filter Options
                dbc.Col([
                    dbc.Card([
                        dbc.CardHeader(html.H4("Filter Options", id="filter-options"), style={'color': '#000000'}),
                        dbc.CardBody([
                            dcc.RadioItems(
                                id='filter-type',
                                options=[
                                    {'label': 'Water Rescue', 'value': 'Water Rescue'},
                                    {'label': 'Mountain Rescue', 'value': 'Mountain or Wilderness Rescue'},
                                    {'label': 'Disaster Rescue', 'value': 'Disaster or Individual Tracking'},
                                    {'label': 'Reset', 'value': 'Reset'}
                                ],
                                value='Reset',
                                labelStyle={'display': 'block', 'margin-bottom': '10px', 'color': '#000000'},
                                style={'font-size': '18px'},
                            ),
                            dbc.Button("Refresh Data", id="refresh-button", color="primary", className="mt-3 me-2"),
                            dbc.Button("Download Data", id="download-button", color="info", className="mt-3")
                        ])
                    ], style={'backgroundColor': '#e7f9c3'})
                ], width=3),

                # Data Table Section
                dbc.Col([
                    dbc.Card([
                        dbc.CardHeader(html.H4("Animal Data", id="animal-data"), style={'color': '#000000'}),
                        dbc.CardBody([
                            dash_table.DataTable(
                                id='datatable-id',
                                columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                                data=df.to_dict('records'),
                                editable=True,
                                row_selectable="single",
                                filter_action="native",
                                sort_action="native",
                                page_action="native",
                                page_current=0,
                                page_size=20,
                                style_table={'overflowX': 'auto', 'height': '400px', 'minWidth': '100%'},
                                style_cell={
                                    'textAlign': 'left',
                                    'padding': '10px',
                                    'fontFamily': 'Arial, sans-serif',
                                    'fontSize': '14px',
                                    'backgroundColor': '#ffffff',
                                    'color': '#000000',
                                },
                                style_header={
                                    'backgroundColor': '#343a40',
                                    'color': '#ffffff',
                                    'fontWeight': 'bold',
                                    'border': '1px solid #333'
                                },
                                style_data_conditional=[
                                    {'if': {'row_index': 'odd'}, 'backgroundColor': '#f9f9f9'},
                                    {'if': {'row_index': 'even'}, 'backgroundColor': '#ffffff'},
                                    {'if': {'state': 'selected'}, 'backgroundColor': '#D2F3FF', 'border': '1px solid #0074D9'}
                                ]
                            )
                        ])
                    ], style={'backgroundColor': '#f7df34'})
                ], width=9)
            ], className="mb-4"),

            # Charts and Map Section
            dbc.Row([
                dbc.Col([dcc.Graph(id="graph-id")], width=6),
                dbc.Col([dl.Map(
                    id="map-id",
                    style={'width': '100%', 'height': '400px'},
                    zoom=10,
                    center=(30.75, -97.48),
                    children=[dl.TileLayer()]
                )], width=6)
            ], className="mb-4"),

            # About Us and Contact Us Sections
html.Div(
    id="info-section",
    children=[
        dbc.Row([
            dbc.Col([
                html.Div([
                    html.H3("About Us", id="about-us", style={'color': '#ffffff'}),
                    html.P(
                        "At EJG's Animal Shelter Dashboard, we specialize in creating innovative solutions for animal rescue organizations. "
                        "Our application is designed to simplify the management of animal data, track rescue operations, and streamline adoption processes. "
                        "With an intuitive interface, users can easily search, filter, and analyze animal records while gaining valuable insights through visual dashboards and maps. "
                        "Security is at the core of what we do, featuring advanced safeguards like Multi-Factor Authentication (MFA), password encryption, and role-based access control so that your sensitive information is protected. "
                        "Our goal is to empower shelters with the tools they need to operate efficiently and focus on their mission of saving lives."
                    )
                ], style={'backgroundColor': '#a013eb', 'padding': '20px', 'borderRadius': '10px'})
            ])
        ], className="mt-5 mb-5"),
        dbc.Row([
            dbc.Col([
                html.Div([
                    html.H3("Contact Us", id="contact", style={'color': '#ffffff'}),
                    html.P(
                        "For more information or assistance, please contact Edward Garcia at edward@example.com. "
                    )
                ], style={'backgroundColor': '#a013eb', 'padding': '20px', 'borderRadius': '10px'})
            ])
        ], className="mt-5 mb-5"),
    ]
)
        ],
        style={'display': 'none'} if not user_authenticated else {'display': 'block'}
    ),
    dcc.Download(id="download-dataframe-csv")  # CSV Download Component
], fluid=True)

#############################################
# Interaction Between Components / Controller
#############################################

# Here I added a callback to filter data based on search input or filter type.
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value'), Input('refresh-button', 'n_clicks'), Input('search-input', 'value')]
)
def update_dashboard(filter_type, n_clicks, search_value):
    # Here I added search functionality in the navigation bar to filter by breed or name.
    # I also used `bypass_cache=True` so that the database is queried directly, ignoring cached results.
    if filter_type == 'Water Rescue':
        dff = shelter.read({"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}}, bypass_cache=True)
    elif filter_type == 'Mountain or Wilderness Rescue':
        dff = shelter.read({"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Border Collie", "Siberian Husky"]}}, bypass_cache=True)
    elif filter_type == 'Disaster or Individual Tracking':
        dff = shelter.read({"breed": {"$in": ["Doberman Pinscher", "Bloodhound", "Rottweiler"]}}, bypass_cache=True)
    else:
        dff = shelter.read({}, bypass_cache=True)

    df_filtered = pd.DataFrame.from_records(dff)

    # Here I dropped the '_id' column from the filtered data if it exists.
    if '_id' in df_filtered.columns:
        df_filtered.drop(columns=['_id'], inplace=True)

    # Here I applied a search filter if a search value is provided to match animal names or breeds.
    # I also made the search case insensitive and applied data handling techniques.
    if search_value:
        search_value = search_value.lower()  
        df_filtered = df_filtered[df_filtered.apply(
            lambda row: search_value in row['breed'].lower() or search_value in str(row['name']).lower(), axis=1
        )]

    return df_filtered.to_dict('records')

# Here I added a callback to download data as a CSV file.
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("download-button", "n_clicks"),
    prevent_initial_call=True
)
def download_data(n_clicks):
    # Here I implemented CSV download functionality for ease of data access.
    return dcc.send_data_frame(df.to_csv, "animal_shelter_data.csv")

# Here I added a callback to update the pie chart based on the filtered data.
@app.callback(
    Output('graph-id', "figure"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graph(viewData):
    if viewData:
        dff = pd.DataFrame.from_dict(viewData)
    else:
        dff = df

    # Here I created a pie chart showing breed distribution to visualize the data better.
    fig = px.pie(
        dff,
        names='breed',
        title='Breed Distribution'
    )
    
    return fig

# Here I added a unified callback to handle map updates and resizing
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', "derived_virtual_selected_rows"),
     Input('dashboard-section', 'style')],
    [State('datatable-id', 'data')]
)
def update_and_resize_map(selected_rows, dashboard_style, data):
    # If the dashboard is visible, proceed
    if dashboard_style.get('display') == 'block':
        if not selected_rows or not data:
            # Default marker position
            markerArray = (30.75, -97.48)
            toolTip = "Austin Animal Center"
            popUpHeading = "Austin Animal Center"
            popUpParagraph = "Shelter Home Location"
        else:
            # Update marker based on selected row
            dff = pd.DataFrame(data)  # Convert the datatable data to a dataframe
            selected = dff.iloc[selected_rows[0]]
            markerArray = (selected['location_lat'], selected['location_long'])
            toolTip = selected['breed']
            popUpHeading = "Animal Name"
            popUpParagraph = selected['name']

        # Return the updated map component
        return dl.Map(
            style={'width': '100%', 'height': '400px'},
            zoom=10,
            center=markerArray,
            children=[
                dl.TileLayer(),
                dl.Marker(
                    position=markerArray,
                    children=[
                        dl.Tooltip(toolTip),
                        dl.Popup([html.H3(popUpHeading), html.P(popUpParagraph)])
                    ]
                )
            ]
        )
    # If the dashboard is not visible, return nothing
    return None

# Here I added a callback to handle user login and logout.
# This callback manages the visibility of the login and dashboard sections based on the user's authentication status.
# It also validates login credentials, including username, password, and optional OTP for MFA.

@app.callback(
    [
        # Here I defined the output for showing or hiding the login section.
        Output("login-section", "style"),
        # Here I defined the output for showing or hiding the dashboard section.
        Output("dashboard-section", "style"),
        # Here I defined the output to display error messages during login attempts.
        Output("login-error", "children")
    ],
    [
        # Here I added an input to handle clicks on the login button.
        Input("login-button", "n_clicks"),
        # Here I added an input to handle clicks on the logout button.
        Input("logout-button", "n_clicks")
    ],
    [
        # Here I captured the username entered by the user in the login form.
        State("username", "value"),
        # Here I captured the password entered by the user in the login form.
        State("password", "value"),
        # Here I captured the OTP entered by the user for MFA in the login form.
        State("otp", "value")
    ]
)
def handle_authentication(login_clicks, logout_clicks, username, password, otp):
    # Here I declared global variables to manage user authentication status and the currently logged-in user.
    global user_authenticated, current_user

    # Here I used `dash.callback_context` to determine which input triggered the callback.
    ctx = dash.callback_context
    if not ctx.triggered:
        # If no input triggered the callback, prevent unnecessary updates.
        raise PreventUpdate

    # Here I identified which button (login or logout) triggered the callback.
    triggered_id = ctx.triggered[0]['prop_id'].split('.')[0]

    # Logout logic
    # Here I added functionality to handle logout requests, resetting authentication and showing the login section.
    if triggered_id == "logout-button":
        user_authenticated = False  # Reset authentication status to unauthenticated.
        current_user = None  # Clear the current user.
        user_management_logger.info("User logged out.")  # Log the logout event.
        return {'display': 'block'}, {'display': 'none'}, ""  # Show login section and hide the dashboard.

    # Login logic
    # Here I added functionality to handle login requests, including validation of username, password, and OTP.
    if triggered_id == "login-button":
        try:
            # Here I checked if the user provided both username and password.
            if username and password:
                # Here I used the UserManagement class to authenticate the user's credentials.
                result = user_mgmt.authenticate_user(username, password, otp)
                if result["status"] == "success":
                    # If authentication is successful, update global variables and show the dashboard.
                    user_authenticated = True
                    current_user = username
                    user_management_logger.info(f"User {username} logged in successfully.")  # Log the successful login event.
                    return {'display': 'none'}, {'display': 'block'}, ""  # Hide login section and show the dashboard.
                else:
                    # If authentication fails, log a warning and display the error message to the user.
                    user_management_logger.warning(f"Failed login attempt for user {username}: {result.get('message')}")
                    return {'display': 'block'}, {'display': 'none'}, result.get("message", "Invalid credentials or OTP.")
            else:
                # If username or password is missing, prompt the user to enter the required details.
                return {'display': 'block'}, {'display': 'none'}, "Please enter username, password, and OTP (if required)."
        except Exception as e:
            # Here I handled any unexpected errors during the login process and logged the error.
            user_management_logger.error(f"Error during login: {str(e)}")
            return {'display': 'block'}, {'display': 'none'}, "An error occurred. Please try again."

    # Here I prevented updates if no relevant button was clicked.
    raise PreventUpdate

    
# Run the Dash app in inline mode
app.run_server(debug=True)

# Print statement to display the URL to the user.
print("Dashboard is running. Go to: http://127.0.0.1:8050")


Dashboard is running. Go to: http://127.0.0.1:8050


In [19]:
# LRU Caching Test for Optimized Query Performance

# This test validates my implementation of LRU (Least Recently Used) caching in the `read` method of the `AnimalShelter` class

import time

# Define a query to test LRU caching
query = {"breed": "Siamese Mix"}

# First query to populate the cache
start_time = time.time()
result1 = shelter.read(query) 
end_time = time.time()
print(f"First query (no cache): {end_time - start_time:.6f} seconds, Results: {len(result1)}")

# Second query which is cached
start_time = time.time()
result2 = shelter.read(query) 
end_time = time.time()
print(f"Second query (cached): {end_time - start_time:.6f} seconds, Results: {len(result2)}")

# Verify that caching gives the same result
assert result1 == result2, "LRU cache did not return the same result."
print("LRU caching is working correctly.")


First query (no cache): 0.009132 seconds, Results: 97
Second query (cached): 0.000514 seconds, Results: 97
LRU caching is working correctly.


In [5]:
# Hash Map Lookup Time Complexity Test

# This test compares the efficiency of hash map lookups against direct database queries.

import time

# Example breed to lookup
breed_to_lookup = "Pit Bull Mix"

# Hash map lookup
start_time = time.time()
hash_map_results = shelter.breed_hash_map.get(breed_to_lookup, [])
end_time = time.time()
print(f"Hash map lookup time: {end_time - start_time:.6f} seconds, Results: {len(hash_map_results)}")

# Direct database query for comparison
start_time = time.time()
db_results = list(shelter.collection.find({"breed": breed_to_lookup}))
end_time = time.time()
print(f"Direct database query time: {end_time - start_time:.6f} seconds, Results: {len(db_results)}")


Hash map lookup time: 0.000000 seconds, Results: 801
Direct database query time: 0.071449 seconds, Results: 801


In [8]:
# LRU Cache Hit & Miss Test

# This test shows the effectiveness of the LRU caching mechanism by monitoring cache hits and misses. 

from functools import lru_cache

# Query to test
query = {"breed": "Siberian Husky Mix"}

# Perform the first read 
shelter.clear_cache()  
result1 = shelter.read(query)
print(f"Cache Info After First Query: {shelter._cached_read.cache_info()}")

# Perform the second read 
result2 = shelter.read(query)
print(f"Cache Info After Second Query: {shelter._cached_read.cache_info()}")


Cache Info After First Query: CacheInfo(hits=0, misses=1, maxsize=128, currsize=1)
Cache Info After Second Query: CacheInfo(hits=1, misses=1, maxsize=128, currsize=1)


In [11]:
# Binary Search Verification for Animal Type Test

# Thist test validates binary search functionality on sorted unique animal types retrieved from the database.

# Query all unique `animal_type` values from the database
animal_types = sorted(
    set(doc["animal_type"] for doc in shelter.collection.find({}, {"animal_type": 1, "_id": 0}) if "animal_type" in doc)
)

print(f"Unique Animal Types (Sorted): {animal_types}")

# Define the target animal type for the binary search, I chose dog as an example
target_animal_type = "Dog"

# Perform binary search
low, high = 0, len(animal_types) - 1
found = False

while low <= high:
    mid = (low + high) // 2
    if animal_types[mid] == target_animal_type:
        found = True
        break
    elif animal_types[mid] < target_animal_type:
        low = mid + 1
    else:
        high = mid - 1

# Print the result
print(f"Binary search for animal type '{target_animal_type}': {'Found' if found else 'Not Found'}")


Unique Animal Types (Sorted): ['Bird', 'Cat', 'Dog', 'Livestock', 'Other']
Binary search for animal type 'Dog': Found


In [13]:
# Data Integrity Test

# Verifies consistency between the hash map and database by comparing breed counts 

# Get all unique breeds from the database
db_breeds = shelter.collection.distinct("breed")

# Verify hash map consistency
for breed in db_breeds:
    hash_map_count = len(shelter.breed_hash_map.get(breed, []))
    db_count = shelter.collection.count_documents({"breed": breed})
    assert hash_map_count == db_count, f"Mismatch for breed '{breed}': Hash map({hash_map_count}) vs DB({db_count})"

print("Hash map and database are consistent for all breeds.")


Hash map and database are consistent for all breeds.


In [15]:
# Cache Performance Test 

# This test compares the query performance with and without the LRU cache implementation. 

import time

# Query for performance testing
query = {"breed": "American Bulldog"}

# Without cache
shelter.clear_cache()  # Clear cache
start_time = time.time()
result_no_cache = shelter.read(query, bypass_cache=True)
end_time = time.time()
print(f"Query without cache: {end_time - start_time:.6f} seconds, Results: {len(result_no_cache)}")

# With cache
start_time = time.time()
result_with_cache = shelter.read(query)
end_time = time.time()
print(f"Query with cache: {end_time - start_time:.6f} seconds, Results: {len(result_with_cache)}")


Query without cache: 0.011664 seconds, Results: 4
Query with cache: 0.004052 seconds, Results: 4


In [17]:
# Hashmap Memory Usage Test 

# This test evaluates the memory footprint of the hash map used for optimized lookups
# then compares it with the size of the database.

import sys

# Calculate memory usage of the hash map
hash_map_memory = sys.getsizeof(shelter.breed_hash_map)
print(f"Memory usage of hash map: {hash_map_memory / 1024:.2f} KB")

# Compare with database size
db_count = shelter.collection.estimated_document_count()
print(f"Database document count: {db_count}")


Memory usage of hash map: 25.42 KB
Database document count: 10000
